<a href="https://colab.research.google.com/github/Servat0r/ISPR-Midterms-2023/blob/master/Midterm3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Midterm 3 ISPR 2023 - Assignment 2 - Salvatore Correnti (matr. 584136)
In this assignment we will design and test on the `CIFAR-10` dataset a custom Convolutional Neural Network.

## Initial Imports
As usual, we start with a couple of cells for changing working directory and for all necessary imports before actually starting coding.

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/ISPR-Midterms-2023"

/content/drive/MyDrive/Colab Notebooks/ISPR-Midterms-2023


In [ ]:
# Below is just to make sure we can build Tensorflow with GPU and to avoid a verbose output for installation
!pip install tensorflow 1> /dev/null

Impossibile trovare il percorso specificato.


In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical

## CIFAR-10 Dataset
For training and evaluating our Convolutional Neural Network, we will use the `CIFAR-10` dataset, which is made up of $50,000$ train and $10,000$ test $32 \times 32$ RGB images, and is available in `keras` as a "built-in" dataset.

In [ ]:
(x_dev, y_dev), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Show an image as an example
exampleImage = Image.fromarray(x_dev[0], mode='RGB')
exampleImage = exampleImage.resize((64, 64))  # Just to show it better
exampleImage.show()

We now convert CIFAR-10 labels into `one-hot` format for usage with a "Sotfmax-based" CNN classifier.

In [ ]:
y_dev = to_categorical(y_dev)
y_test = to_categorical(y_test)
y_dev[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

Before proceeding, we also normalize pixel values into the range $[0, 1]$ and we split training data into properly training ones and validation ones (we will keep test data to evaluate the final model).

In [ ]:
x_dev = x_dev.astype(np.float32) / 255
x_test = x_test.astype(np.float32) / 255

from sklearn.model_selection import train_test_split

x_train, x_eval, y_train, y_eval = train_test_split(x_dev, y_dev, test_size=0.2, random_state=0)

## Designing a Convolutional Neural Network for CIFAR-10
When designing a custom Convolutional Neural Network for a specific task, it is important to take into account at least the most important "building blocks" for CNNs that have been developed through the years and used in "reference" models. Indeed, since CNNs are in usage by about a decade, it is convenient to take inspiration or directly modify one of these reference models to achieve our objectives. Also, monitoring the accuracy and the space and time required for training a given model (e.g. by calculating the number of parameters) is also fundamental, and in particular since we don't have required hardware for huge models we want to get a tradeoff between accuracy and size of the model.

We will then go through our discussion throughout the following design choices:

1. `Target Accuracy`: we want to achieve an accuracy at least $\geq 70\%$, since this is the accuracy that one can easily get with a "reduced" `VGG` model (see below). On top of that, we can consider additional design choices based on what we get from experiments for achieving a higher one;
2. `"Style" of the CNN`: we will use the "traditional" design pattern of a series of Convolutional-MaxPooling layers, ended by a `Fully-Connected` block and `Softmax` activation, as employed in `AlexNet`, `VGG` and (apart from MaxPool) in `GoogLeNet`;
3. `Base Model`: since `CIFAR-10` images are of size $32 \times 32$, we are not interested in very deep networks for reducing the feature maps sizes up to reasonable values for usage with a final sequence of fully-connected layers, or in other words if we use `max-pooling` with a pool size of $(2, 2)$, it suffices to employ $3$ `MaxPool2D` layers to get a feature map of size $\leq 4 \times 4$, and if we suppose to have $N$ filters at the end, we will get $16N$ input units for the `dense` part of the network, which can be reasonable if we take for example a single hidden layer with a size $\leq 128$, or we directly skip the `dense hidden` layers. As a consequence, we will model our CNNs as a sort of "reduced" version of `VGG`, which has proved to be quite effective in classification tasks over `ImageNet`;
4. `Number of Parameters`: ideally, we want to keep $< 1,000,000$ parameters for our Convolutional Neural Network, which is suitable for a 2-deep or 3-deep (in the sense of Conv2D-MaxPool blocks) `VGG-like` network. After having built a "satisfactory" network without explicit design choices for restricting the number of parameters, we can explore usage of $1 \times 1$ convolutions and reduced `Dense` blocks to reduce the number of parameters while retaining most of the accuracy;
5. `Regularization`: since a `VGG-like` CNN can quickly become quite big, especially in the `Dense` part, it is essential to adopt regularization strategies to limit overfitting and improving overall performance. From an architectural point of view, two viable yet effective strategies are `Dropout` and `Batch Normalization`, and we will experiment with both of them to see if we can improve overall performance of the network;
6. `Advanced Blocks`: if we manage to keep our network "sufficiently small" (i.e. with 2 or 3 Conv2D-MaxPool blocks), we may not use advanced architectural patterns like `Skip Connections` and `Inception Blocks`, which complicate the design and coding of the Neural Network, especially if we can keep under control `vanishing gradient` phenomena by avoiding a too deep network.

### VGG Network
<img src="vgg16.png">

**`VGG16`** (**`Visual Geometry Group 16`**) network was developed in (?) by (?) and won the (?) competition on the `ImageNet` dataset. As we can see in the above figure, VGG16 is composed by a sequence of $5$ `Conv2D-Conv2D-MaxPool` blocks, i.e. 2 Convolutional Layers with `same` padding and $3 \times 3$ kernel sizes, followed by a single MaxPooling layer with pool size of $2 \times 2$ for reducing feature maps sizes. As we can see, each block contains 2 Convolutional Layer with an increasing number of filters for each one ($64$, $128$, $256$, $512$, $512$) and a single hidden layer of size $4096$.

The idea behind this pattern is that by reducing the size of the feature maps we are progressively representing "higher-level" features that somehow "summarize" information by lower-level ones, hence the number of "descriptors" we want to keep should be increasing when we traverse the network, and moreover we want to let "feature descriptors" from more and more distant areas of the image to combine themselves.

#### Reduced VGG
We can keep the above design pattern in our CNN by simply reducing the number of Blocks and the size of the hidden layer, or by removing it at all. For example, we can employ a first block made up of two convolutional layers each one with $32$ filters of size $3 \times 3$, a second one with $64$ filters and a third one of $128$ ones. As in VGG16, we use `same` padding even if it may introduce some form of bias due to the padding value since we are using a very small kernel.

We will start with a "simple" Reduced VGG without any Dropout, Batch Normalization or Dense Hidden Layer to see if this is a viable baseline.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization

In [ ]:
baseReducedVGG = Sequential()

baseReducedVGG.add(Conv2D(32, kernel_size=3, activation="relu", input_shape=(32,32,3), padding='same'))
baseReducedVGG.add(Conv2D(32, kernel_size=3, activation="relu", padding='same'))
baseReducedVGG.add(MaxPool2D())

baseReducedVGG.add(Conv2D(64, kernel_size=3, activation="relu", padding='same'))
baseReducedVGG.add(Conv2D(64, kernel_size=3, activation="relu", padding='same'))
baseReducedVGG.add(MaxPool2D())

baseReducedVGG.add(Conv2D(128, kernel_size=3, activation="relu", padding='same'))
baseReducedVGG.add(Conv2D(128, kernel_size=3, activation="relu", padding='same'))
baseReducedVGG.add(MaxPool2D())

# Now we flatten for fully-connected part
baseReducedVGG.add(Flatten())
baseReducedVGG.add(Dense(10, activation="softmax"))
baseReducedVGG.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                    

As we can see, our `baseReducedVGG` model has $307,498$ parameters, which is a reasonable amount and far below the $1,000,000$ maximum we mentioned before. We also notice that since we have not used any Dense hidden layer, the two Conv2D layers with $128$ filters contribute for $\approx 75\%$ of the total parameters, hence we will not use any higher number of filters to keep number of parameters (hence training time) under control.

We now compile and train the model with a batch size of $64$ and for $10$ epochs, using `Adam` optimizer and `Categorical Cross Entropy` loss:

In [ ]:
baseReducedVGG.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
baseReducedVGGHistory = baseReducedVGG.fit(
    x_train, y_train, validation_data=(x_eval, y_eval), epochs=10, batch_size=64,
    callbacks=[tf.keras.callbacks.CSVLogger('baseReducedVGG_log.csv')],
)

Epoch 1/10
625/625 [==============================] - 78s 123ms/step - loss: 1.5676 - accuracy: 0.4262 - val_loss: 1.2576 - val_accuracy: 0.5510
Epoch 2/10
625/625 [==============================] - 77s 123ms/step - loss: 1.0678 - accuracy: 0.6219 - val_loss: 0.9501 - val_accuracy: 0.6671
Epoch 3/10
625/625 [==============================] - 77s 123ms/step - loss: 0.8487 - accuracy: 0.7031 - val_loss: 0.8160 - val_accuracy: 0.7160
Epoch 4/10
625/625 [==============================] - 81s 129ms/step - loss: 0.6961 - accuracy: 0.7577 - val_loss: 0.7427 - val_accuracy: 0.7410
Epoch 5/10
625/625 [==============================] - 81s 130ms/step - loss: 0.5838 - accuracy: 0.7975 - val_loss: 0.7546 - val_accuracy: 0.7457
Epoch 6/10
625/625 [==============================] - 83s 132ms/step - loss: 0.4889 - accuracy: 0.8299 - val_loss: 0.7460 - val_accuracy: 0.7543
Epoch 7/10
625/625 [==============================] - 82s 131ms/step - loss: 0.3984 - accuracy: 0.8624 - val_loss: 0.7304 - val_ac

As we can see, we already got a very high training accuracy ($92.87\%$) and a quite high validation one ($76.58\%$). From the results of the last $3$ epochs we also notice that validation accuracy is substantially stalling in the interval $[0.75, 0.77]$, so we may imagine that we are reaching an overfitting point. As already said, this test is to actually verify that a "base" ReducedVGG actually works well with CIFAR-10. We now move onto adding Dropout and Batch Normalization.

#### Adding Dropout and Batch Normalization
We will add a `BatchNormalization` layer after each Convolutional, MaxPooling and non-output Dense layer, since we want to avoid each possible "internal shift" in the distribution that the network is learning. By contrary, we can add Dropout layers after each Conv2D-MaxPool block, since we do not have any trainable parameter in Dropout layers and moreover its effect as regularizer is actually to train an implicit emsembler of subnetworks inside the original one, and we do not want to "fragment" too much the network; in particular, we want to keep Conv2D-MaxPool blocks as "units" for the underlying subnetworks.

We do not want to do any hyperparameter tuning by now, hence we set Dropout probabilities to standard values of $0.2$ and $0.3$, paying attention to the fact that as we increase the number of filters and hence the number of parameters we want a higher regularization effect, hence a higher Dropout probability.

In [ ]:
regularizedReducedVGG = Sequential()

regularizedReducedVGG.add(Conv2D(32, kernel_size=3, activation="relu", input_shape=(32,32,3), padding='same'))
regularizedReducedVGG.add(BatchNormalization())
regularizedReducedVGG.add(Conv2D(32, kernel_size=3, activation="relu", padding='same'))
regularizedReducedVGG.add(BatchNormalization())
regularizedReducedVGG.add(MaxPool2D())
regularizedReducedVGG.add(BatchNormalization())
regularizedReducedVGG.add(Dropout(0.2))

regularizedReducedVGG.add(Conv2D(64, kernel_size=3, activation="relu", padding='same'))
regularizedReducedVGG.add(BatchNormalization())
regularizedReducedVGG.add(Conv2D(64, kernel_size=3, activation="relu", padding='same'))
regularizedReducedVGG.add(BatchNormalization())
regularizedReducedVGG.add(MaxPool2D())
regularizedReducedVGG.add(BatchNormalization())
regularizedReducedVGG.add(Dropout(0.3))

regularizedReducedVGG.add(Conv2D(128, kernel_size=3, activation="relu", padding='same'))
regularizedReducedVGG.add(BatchNormalization())
regularizedReducedVGG.add(Conv2D(128, kernel_size=3, activation="relu", padding='same'))
regularizedReducedVGG.add(BatchNormalization())
regularizedReducedVGG.add(MaxPool2D())
regularizedReducedVGG.add(BatchNormalization())
regularizedReducedVGG.add(Dropout(0.3))

regularizedReducedVGG.add(Flatten())
regularizedReducedVGG.add(BatchNormalization())
regularizedReducedVGG.add(Dense(10, activation="softmax"))

regularizedReducedVGG.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                        

In [ ]:
regularizedReducedVGG.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
regularizedReducedVGGHistory = regularizedReducedVGG.fit(
    x_train, y_train, validation_data=(x_eval, y_eval), epochs=10, batch_size=64,
    callbacks=[tf.keras.callbacks.CSVLogger('baseReducedVGG_log.csv')],
)

Epoch 1/10
625/625 [==============================] - 109s 171ms/step - loss: 1.5031 - accuracy: 0.4895 - val_loss: 1.1002 - val_accuracy: 0.6169
Epoch 2/10
625/625 [==============================] - 110s 176ms/step - loss: 1.0239 - accuracy: 0.6458 - val_loss: 0.8442 - val_accuracy: 0.7082
Epoch 3/10
625/625 [==============================] - 107s 172ms/step - loss: 0.8675 - accuracy: 0.6997 - val_loss: 0.8589 - val_accuracy: 0.7132
Epoch 4/10
625/625 [==============================] - 109s 174ms/step - loss: 0.7718 - accuracy: 0.7336 - val_loss: 0.7473 - val_accuracy: 0.7458
Epoch 5/10
625/625 [==============================] - 106s 170ms/step - loss: 0.7045 - accuracy: 0.7577 - val_loss: 0.7910 - val_accuracy: 0.7366
Epoch 6/10
625/625 [==============================] - 107s 171ms/step - loss: 0.6509 - accuracy: 0.7750 - val_loss: 0.6986 - val_accuracy: 0.7672
Epoch 7/10
625/625 [==============================] - 107s 171ms/step - loss: 0.6002 - accuracy: 0.7927 - val_loss: 0.7227 -

We notice that now final training accuracy if lower than in the previous case ($82.45\%$ against $92.87\%$), but on the bright side we got a small improvement in the validation accuracy of about $1-2\%$, hence we can keep Dropout and BatchNormalization inside our final network.

#### Adding Dense layers
We now try to add `Dense` hidden layers in the final part of the network to see if this improves accuracy. As already seen, after Conv2D-MaxPool blocks we have $2048$ parameters in the flattened array, hence if we insert a hidden layer of size $m$, we get $2058m$ parameters. To keep the total number of parameters $< 1,000,000$, and possibly $< 500,000$ but without forcing the network to "simplify" too much the representation in the flattened array, we can take a hidden size of $128$. As for the previous network, we add a BatchNormalization layer and a Dropout one after the hidden layer and we use a Dropout probability of $0.5$.

In [ ]:
dense128ReducedVGG = Sequential()

dense128ReducedVGG.add(Conv2D(32, kernel_size=3, activation="relu", input_shape=(32,32,3), padding='same'))
dense128ReducedVGG.add(BatchNormalization())
dense128ReducedVGG.add(Conv2D(32, kernel_size=3, activation="relu", padding='same'))
dense128ReducedVGG.add(BatchNormalization())
dense128ReducedVGG.add(MaxPool2D())
dense128ReducedVGG.add(Dropout(0.2))
dense128ReducedVGG.add(BatchNormalization())

dense128ReducedVGG.add(Conv2D(64, kernel_size=3, activation="relu", padding='same'))
dense128ReducedVGG.add(BatchNormalization())
dense128ReducedVGG.add(Conv2D(64, kernel_size=3, activation="relu", padding='same'))
dense128ReducedVGG.add(BatchNormalization())
dense128ReducedVGG.add(MaxPool2D())
dense128ReducedVGG.add(Dropout(0.3))
dense128ReducedVGG.add(BatchNormalization())

dense128ReducedVGG.add(Conv2D(128, kernel_size=3, activation="relu", padding='same'))
dense128ReducedVGG.add(BatchNormalization())
dense128ReducedVGG.add(Conv2D(128, kernel_size=3, activation="relu", padding='same'))
dense128ReducedVGG.add(BatchNormalization())
dense128ReducedVGG.add(MaxPool2D())
dense128ReducedVGG.add(Dropout(0.3))
dense128ReducedVGG.add(BatchNormalization())

dense128ReducedVGG.add(Flatten())
dense128ReducedVGG.add(BatchNormalization())
dense128ReducedVGG.add(Dense(128, activation="relu", kernel_initializer="he_uniform", bias_initializer="he_uniform"))
dense128ReducedVGG.add(Dropout(0.5))
dense128ReducedVGG.add(BatchNormalization())
dense128ReducedVGG.add(Dense(10, activation="softmax"))

dense128ReducedVGG.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                        

In [ ]:
dense128ReducedVGG.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
dense128ReducedVGGHistory = dense128ReducedVGG.fit(
    x_train, y_train, validation_data=(x_eval, y_eval), epochs=10, batch_size=64,
    callbacks=[tf.keras.callbacks.CSVLogger('baseReducedVGG_log.csv')],
)

Epoch 1/10
625/625 [==============================] - 109s 170ms/step - loss: 1.5771 - accuracy: 0.4438 - val_loss: 1.3246 - val_accuracy: 0.5331
Epoch 2/10
625/625 [==============================] - 107s 170ms/step - loss: 1.0837 - accuracy: 0.6162 - val_loss: 0.8731 - val_accuracy: 0.6914
Epoch 3/10
625/625 [==============================] - 113s 182ms/step - loss: 0.9240 - accuracy: 0.6762 - val_loss: 0.8488 - val_accuracy: 0.6973
Epoch 4/10
625/625 [==============================] - 115s 184ms/step - loss: 0.8226 - accuracy: 0.7140 - val_loss: 0.6994 - val_accuracy: 0.7535
Epoch 5/10
625/625 [==============================] - 112s 179ms/step - loss: 0.7564 - accuracy: 0.7364 - val_loss: 0.6965 - val_accuracy: 0.7575
Epoch 6/10
625/625 [==============================] - 112s 179ms/step - loss: 0.6922 - accuracy: 0.7615 - val_loss: 0.6474 - val_accuracy: 0.7797
Epoch 7/10
625/625 [==============================] - 112s 180ms/step - loss: 0.6450 - accuracy: 0.7767 - val_loss: 0.6834 -

As we can see, we got an improvement in validation accuracy of about $5\%$, while training one is stable. We can then keep the hidden Dense layer of size $128$ and test the final model on the Test Set.

We now evaluate the model over the Test Set:

To summarize, our custom Convolutional Neural Network has the following structure:
* A sequence of two **Convolutional Layers** with $32$ filters of size $3 \times 3$ with `same` padding and **Batch Normalization** after each layer;
* A **MaxPool** layer of pooling size of $2 \times 2$, followed by a **Dropout** layer with a (default) dropout probability of $0.2$ and another **Batch Normalization** one;
* A sequence of two **Convolutional Layers** with $64$ filters of size $3 \times 3$ with `same` padding and **Batch Normalization** after each layer;
* A **MaxPool** layer of pooling size of $2 \times 2$, followed by a **Dropout** layer with a (default) dropout probability of $0.3$ and another **Batch Normalization** one;
* A sequence of two **Convolutional Layers** with $128$ filters of size $3 \times 3$ with `same` padding and **Batch Normalization** after each layer;
* A **MaxPool** layer of pooling size of $2 \times 2$, followed by a **Dropout** layer with a (default) dropout probability of $0.3$ and another **Batch Normalization** one;
* After that, the output is flattened to an array of size $2048$ and forwarded to a **Dense** layer of size $2048 \times 128$;
* Finally, we have another **Dense** layer of size $128 \times 10$ and a **Softmax** layer for outputting probability distributions over the $10$ classes for the given input.

Our network is a sort of `"ReducedVGG"` in the sense that follows the architectural pattern of `VGG16`, but it adapts it to the smaller size of `CIFAR-10` images and introduces `Dropout` and `BatchNormalization` for having regularization and robustness against internal distribution shifts. We have used neither any Skip or Residual Connections nor any `Inception` module to keep the model simple ($561,962$ parameters) while still retaining a good accuracy in the range $80-82\%$.

In [ ]:
dense256ReducedVGG = Sequential()

dense256ReducedVGG.add(Conv2D(32, kernel_size=3, activation="relu", input_shape=(32, 32, 3), padding='same'))
dense256ReducedVGG.add(BatchNormalization())
dense256ReducedVGG.add(Conv2D(32, kernel_size=3, activation="relu", padding='same'))
dense256ReducedVGG.add(BatchNormalization())
dense256ReducedVGG.add(MaxPool2D())
dense256ReducedVGG.add(Dropout(0.3))
dense256ReducedVGG.add(BatchNormalization())

dense256ReducedVGG.add(Conv2D(64, kernel_size=3, activation="relu", padding='same'))
dense256ReducedVGG.add(BatchNormalization())
dense256ReducedVGG.add(Conv2D(64, kernel_size=3, activation="relu", padding='same'))
dense256ReducedVGG.add(BatchNormalization())
dense256ReducedVGG.add(MaxPool2D())
dense256ReducedVGG.add(Dropout(0.3))
dense256ReducedVGG.add(BatchNormalization())

dense256ReducedVGG.add(Conv2D(128, kernel_size=3, activation="relu", padding='same'))
dense256ReducedVGG.add(BatchNormalization())
dense256ReducedVGG.add(Conv2D(128, kernel_size=3, activation="relu", padding='same'))
dense256ReducedVGG.add(BatchNormalization())
dense256ReducedVGG.add(MaxPool2D())
dense256ReducedVGG.add(Dropout(0.3))
dense256ReducedVGG.add(BatchNormalization())

dense256ReducedVGG.add(Conv2D(128, kernel_size=3, activation="relu", padding='same'))
dense256ReducedVGG.add(BatchNormalization())
dense256ReducedVGG.add(Conv2D(128, kernel_size=3, activation="relu", padding='same'))
dense256ReducedVGG.add(BatchNormalization())
dense256ReducedVGG.add(MaxPool2D())
dense256ReducedVGG.add(Dropout(0.3))
dense256ReducedVGG.add(BatchNormalization())

dense256ReducedVGG.add(Flatten())
dense256ReducedVGG.add(BatchNormalization())
dense256ReducedVGG.add(Dense(256, activation="relu", kernel_initializer="he_uniform", bias_initializer="he_uniform"))
dense256ReducedVGG.add(Dropout(0.5))
dense256ReducedVGG.add(BatchNormalization())
dense256ReducedVGG.add(Dense(10, activation="softmax"))

dense256ReducedVGG.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_72 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_41 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_73 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                            
                                                      

In [ ]:
dense256ReducedVGG.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
dense256ReducedVGGHistory = dense256ReducedVGG.fit(
    x_train, y_train, validation_data=(x_eval, y_eval), epochs=10, batch_size=64,
    callbacks=[tf.keras.callbacks.CSVLogger('baseReducedVGG_log.csv')],
)

Epoch 1/10
625/625 [==============================] - 121s 187ms/step - loss: 1.7213 - accuracy: 0.3919 - val_loss: 1.4986 - val_accuracy: 0.4742
Epoch 2/10
625/625 [==============================] - 112s 179ms/step - loss: 1.1622 - accuracy: 0.5860 - val_loss: 1.0086 - val_accuracy: 0.6326
Epoch 3/10
625/625 [==============================] - 111s 177ms/step - loss: 0.9627 - accuracy: 0.6603 - val_loss: 0.9091 - val_accuracy: 0.6848
Epoch 4/10
625/625 [==============================] - 112s 179ms/step - loss: 0.8336 - accuracy: 0.7133 - val_loss: 0.9559 - val_accuracy: 0.6662
Epoch 5/10
625/625 [==============================] - 112s 179ms/step - loss: 0.7486 - accuracy: 0.7434 - val_loss: 0.7616 - val_accuracy: 0.7412
Epoch 6/10
625/625 [==============================] - 111s 178ms/step - loss: 0.6825 - accuracy: 0.7646 - val_loss: 0.6516 - val_accuracy: 0.7669
Epoch 7/10
625/625 [==============================] - 118s 188ms/step - loss: 0.6308 - accuracy: 0.7844 - val_loss: 0.6961 -

In [ ]:
dense256ReducedVGG.fit(
    x_train, y_train, validation_data=(x_eval, y_eval), epochs=3, batch_size=64,
    callbacks=[tf.keras.callbacks.CSVLogger('baseReducedVGG_log.csv')],
)

Epoch 1/3
625/625 [==============================] - 114s 183ms/step - loss: 0.4790 - accuracy: 0.8370 - val_loss: 0.5723 - val_accuracy: 0.8069
Epoch 2/3
625/625 [==============================] - 114s 183ms/step - loss: 0.4439 - accuracy: 0.8476 - val_loss: 0.5347 - val_accuracy: 0.8163
Epoch 3/3
625/625 [==============================] - 115s 184ms/step - loss: 0.4177 - accuracy: 0.8556 - val_loss: 0.5175 - val_accuracy: 0.8265


In [ ]:
dense256ReducedVGG.fit(
    x_train, y_train, validation_data=(x_eval, y_eval), epochs=3, batch_size=64,
    callbacks=[tf.keras.callbacks.CSVLogger('baseReducedVGG_log.csv')],
)

Epoch 1/3
625/625 [==============================] - 126s 201ms/step - loss: 0.3931 - accuracy: 0.8637 - val_loss: 0.4998 - val_accuracy: 0.8343
Epoch 2/3
625/625 [==============================] - 120s 192ms/step - loss: 0.3768 - accuracy: 0.8688 - val_loss: 0.4884 - val_accuracy: 0.8389
Epoch 3/3
625/625 [==============================] - 126s 202ms/step - loss: 0.3597 - accuracy: 0.8771 - val_loss: 0.5435 - val_accuracy: 0.8269
